In [1]:
import pandas as pd
# Cargar el archivo y eliminar filas donde el clúster es NaN
file_path = 'data/data.csv'
data = pd.read_csv(file_path)
data = data.dropna(subset=['Cluster_Label'])

In [2]:

from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Función para preparar los datos para el modelo LSTM
def prepare_data(data, look_back=3):
    features = ['landed_w_kg', '0.49402499_m', '1.541375_m', '2.645669_m', '3.819495_m', 
                '5.0782242_m', '6.4406142_m', '7.9295602_m', '9.5729971_m', 'mean_temp']
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(data[features])
    
    X, y = [], []
    for i in range(len(data_scaled) - look_back):
        a = data_scaled[i:(i + look_back)]
        X.append(a)
        y.append(data_scaled[i + look_back, 0])  # La primera columna es 'landed_w_kg'
    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))
    return X, y

# Filtrar datos para la especie CORVINA
corvina_data = data[data['species'] == 'CORVINA']
X, y = prepare_data(corvina_data)


2024-06-25 18:03:04.266340: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-25 18:03:04.269569: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-25 18:03:04.313506: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-25 18:03:04.313537: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-25 18:03:04.314717: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [3]:
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(25, activation='relu', return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(12, activation='linear'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Crear modelo
input_shape = (X.shape[1], X.shape[2])
model = create_lstm_model(input_shape)

# Configurar EarlyStopping y ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)
checkpoint = ModelCheckpoint('corvina_best_model.h5', monitor='loss', save_best_only=True, mode='min')

# Entrenar el modelo
model.fit(X, y, epochs=50, batch_size=1, verbose=2, callbacks=[early_stopping, checkpoint])


Epoch 1/50
46094/46094 - 130s - loss: 1.9537e-04 - 130s/epoch - 3ms/step
Epoch 2/50


/opt/conda/envs/cedo/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


46094/46094 - 126s - loss: 1.7522e-04 - 126s/epoch - 3ms/step
Epoch 3/50
46094/46094 - 126s - loss: 1.6075e-04 - 126s/epoch - 3ms/step
Epoch 4/50
46094/46094 - 127s - loss: 1.5955e-04 - 127s/epoch - 3ms/step
Epoch 5/50
46094/46094 - 127s - loss: 1.5731e-04 - 127s/epoch - 3ms/step
Epoch 6/50
46094/46094 - 126s - loss: 1.5623e-04 - 126s/epoch - 3ms/step
Epoch 7/50
46094/46094 - 126s - loss: 1.5500e-04 - 126s/epoch - 3ms/step
Epoch 8/50
46094/46094 - 126s - loss: 1.5409e-04 - 126s/epoch - 3ms/step
Epoch 9/50
46094/46094 - 126s - loss: 1.5338e-04 - 126s/epoch - 3ms/step
Epoch 10/50
46094/46094 - 122s - loss: 1.5374e-04 - 122s/epoch - 3ms/step
Epoch 11/50
46094/46094 - 122s - loss: 1.5267e-04 - 122s/epoch - 3ms/step
Epoch 12/50
46094/46094 - 121s - loss: 1.5252e-04 - 121s/epoch - 3ms/step
Epoch 13/50
46094/46094 - 121s - loss: 1.5180e-04 - 121s/epoch - 3ms/step
Epoch 14/50
46094/46094 - 121s - loss: 1.5105e-04 - 121s/epoch - 3ms/step
Epoch 15/50
46094/46094 - 121s - loss: 1.5128e-04 - 121s/

modelo maxplanck tiene 10 realizaciones, los datos proyectados inician en el 2014